In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNetfrom sklearn.preprocessing import StandardScaler


In [ ]:
df = pd.read_csv('CSDataAsCSV.csv', usecols=[i for i in range(6, 18) if i != 16])
df.head()

In [ ]:
scaler = StandardScaler()
scaled_df=df.copy()
scaled_df=pd.DataFrame(scaler.fit_transform(scaled_df), columns=scaled_df.columns)
scaled_df.head()

In [ ]:
X = scaled_df.drop('criticality_score', axis=1)
y = scaled_df['criticality_score']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=101)









1) SelectFromModel









In [ ]:
estimator = Lasso(alpha = 0.1)
featureSelection = SelectFromModel(estimator)
featureSelection.fit(X, y)

In [ ]:
selectedFeatures = featureSelection.transform(X)
selectedFeatures

In [ ]:
X.columns[featureSelection.get_support()]








2) Recursive Feature Selection








In [ ]:
from sklearn.feature_selection import RFE
model = ElasticNet(alpha = 0.01)
rfe = RFE(model)
rfe.fit(X_train, y_train)
print(rfe.support_)
print(rfe.ranking_)

In [ ]:

rfe.score(X_test, y_test)

In [ ]:

pd.DataFrame(rfe.support_,index=X.columns,columns=["Necessary Feature"])

In [ ]:
pd.DataFrame(rfe.ranking_,index=X.columns,columns=["Relative Importance Ranking"])










3) Sequential Forward Floating Selection










In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import KFold

en = ElasticNet(alpha = 0.1)

sfs = SFS(en, 
          k_features=5, 
          forward=True, 
          floating=True, 
          scoring='r2',
          cv = KFold(n_splits=5, shuffle=True))

sfs = sfs.fit(X, y)

In [ ]:
print('\nSequential Forward Floating Selection (k=5):')
print(sfs.k_feature_idx_)
print('CV Score:')
print(sfs.k_score_)

In [ ]:
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
fig = plot_sfs(sfs.get_metric_dict(), kind='std_err')

plt.title('Sequential Forward Selection (w. StdErr)')
plt.grid()
plt.show()

 
 
 
 
 
 
 
 
 
 
 4) Sequential Backward Floating Selection:
 
 
 
 
 
 
 
 
 

In [ ]:
en = ElasticNet(alpha=0.1)

sfs2 = SFS(en, 
          k_features=5, 
          forward=False, 
          floating=True, 
          scoring='r2',
          cv = KFold(n_splits=5, shuffle=True))

sfs2 = sfs2.fit(X, y)

In [ ]:
print('\nSequential Backward Floating Selection (k=5):')
print(sfs2.k_feature_idx_)
print('CV Score:')
print(sfs2.k_score_)